## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [15]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from keras.utils import to_categorical


input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer


'''Resnet 50 架構'''
model=keras.applications.ResNet50(include_top=False, weights='imagenet',
                                  input_tensor=input_tensor,
                                  pooling=None, classes=10)
model.summary()


/home/yanroo/anaconda3/envs/Keras/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

## 添加層數

In [23]:
x = model.output

'''可以參考Cifar10實作章節'''
x = GlobalAveragePooling2D()(x)

predictions = Dense(output_dim=10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))
model.summary()

Model深度： 179
Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
_______________________________________________________________________________

/home/yanroo/anaconda3/envs/Keras/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
  


## 鎖定特定幾層不要更新權重

In [24]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [25]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

(50000, 32, 32, 3)


## Training

In [26]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=100)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
50000/50000 [==============================] - 56s 1ms/step - loss: 2.0612 - acc: 0.2420
Epoch 2/100
50000/50000 [==============================] - 50s 1ms/step - loss: 1.7235 - acc: 0.2983
Epoch 3/100
50000/50000 [==============================] - 50s 1ms/step - loss: 1.6083 - acc: 0.3589
Epoch 4/100
50000/50000 [==============================] - 50s 1ms/step - loss: 1.4580 - acc: 0.4176
Epoch 5/100
50000/50000 [==============================] - 50s 1ms/step - loss: 1.3654 - acc: 0.4639
Epoch 6/100
50000/50000 [==============================] - 50s 1ms/step - loss: 1.2722 - acc: 0.5178
Epoch 7/100
50000/50000 [==============================] - 50s 1ms/step - loss: 1.1947 - acc: 0.5642
Epoch 8/100
50000/50000 [==============================] - 50s 1ms/step - loss: 1.1019 - acc: 0.5976
Epoch 9/100
50000/50000 [==============================] - 50s 1ms/step - loss: 1.0386 - acc: 0.62

50000/50000 [==============================] - 51s 1ms/step - loss: 0.1221 - acc: 0.9695
Epoch 78/100
50000/50000 [==============================] - 51s 1ms/step - loss: 0.1594 - acc: 0.9610
Epoch 79/100
50000/50000 [==============================] - 51s 1ms/step - loss: 0.1260 - acc: 0.9680
Epoch 80/100
50000/50000 [==============================] - 51s 1ms/step - loss: 0.1103 - acc: 0.9721
Epoch 81/100
50000/50000 [==============================] - 51s 1ms/step - loss: 0.1536 - acc: 0.9601
Epoch 82/100
50000/50000 [==============================] - 51s 1ms/step - loss: 0.1094 - acc: 0.9712
Epoch 83/100
50000/50000 [==============================] - 51s 1ms/step - loss: 0.1023 - acc: 0.9740
Epoch 84/100
50000/50000 [==============================] - 51s 1ms/step - loss: 0.0932 - acc: 0.9757
Epoch 85/100
50000/50000 [==============================] - 51s 1ms/step - loss: 0.1013 - acc: 0.9730
Epoch 86/100
50000/50000 [==============================] - 51s 1ms/step - loss: 0.1261 - acc: 